## Import required libraries

In [1]:
import pandas as pd
import re
import requests
import numpy as np
import os.path

## Import raw data

In [2]:
df = pd.read_excel('https://query.data.world/s/slofesjnl4cdvy3jtv6qjyokuqhrjj')
df

,Date,Time,Location,Sub-location,Source,Notes
0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04
1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04
2,2021-06-01,1955h - 2030h,Seletar Mall (33 Sengkang West Avenue),NTUC FairPrice,https://mothership.sg/2021/06/june-4-covid-19-...,Added/ Updated on 2021-06-05
3,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01
4,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02
...,...,...,...,...,...,...
1312,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN
1313,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN
1314,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN
1315,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN


## Get longitude and latitude

In [3]:
request_url = "https://developers.onemap.sg/commonapi/search"

In [4]:
# exceptions
address_changes = {
    '37 & 39 Sultan Gate': '37 Sultan Gate',
    '1 Pasir Ris Drive 3': '1 Pasir Ris Drive',
    '08/09 Marine Parade Central': 'Marine Parade Central',
    '365 & 369 Balestier Road': '365 Balestier Road',
    '1/A Tampines Street 92': '1A Tampines Street 92',
    '1 HabourFront Walk': '1 HarbourFront Walk',
    '1 Jurong West Central 3': '1 Jurong West Central',
    'Block 69   Geylang   Bahru': '69 Geylang Bahru',
    '63-67 Temple Street': '63 Temple Street',
    '612/620 Geylang Lorong 38': '612 Geylang Lorong 38',
    '120 Pasir Ris Central Street 3': '120 Pasir Ris Central',
    '1 Imbiah Road': 'Imbiah Road',
    '163 Geylang Road Lorong 3': '163 Geylang Road',
    'Block 145 Teck Whye Avenue': '145 Teck Whye Avenue',
    '1 Harborfront Walk': '1 HarbourFront Walk',
    '803 King George’s Avenue': '803 King George Avenue',
    '863 Woodlands Street 62': '863 Woodlands',
    '8 Bishan Place': '9 Bishan Place',
    '389 Yung Sheng Road': 'Yung Sheng Road',
    '339 Yung Sheng Road': 'Yung Sheng Road'
}

In [5]:
for index, row in df.iterrows():
    result = re.search('(?s:.*)\((.*?)\)', row['Location'])
    location = result.group(1) if result else location
    location = address_changes.get(location) if location in address_changes else location
    
    # call request
    response = requests.get(request_url, params={
        'searchVal': location,
        'returnGeom': 'Y',
        'getAddrDetails': 'Y'
    })
    print('{}: {}'.format(index, location))
    result = response.json()['results'][0]
    df.at[index,'Longitude']=result['LONGITUDE']
    df.at[index,'Latitude']=result['LATITUDE']
    df.at[index,'Postal']=result['POSTAL']
df

0: 53 Pioneer Road
1: 511 Bukit Batok Street 2
2: 33 Sengkang West Avenue
3: 476D Upper Serangoon View
4: 21 Hougang Street 51
5: 642 Hougang Avenue 8
6: 3 New Bugis Street
7: 38 Beo Crescent
8: 10 Bayfront Avenue
9: 10 Bayfront Avenue
10: 308 Anchorvale Road
11: 33 Sengkang West Avenue
12: 476D Upper Serangoon View
13: 78 Redhill Lane
14: 111 North Bridge Road
15: 7 Pioneer Road North
16: 1 HarbourFront Walk
17: 750A Chai Chee Road
18: 763 Jurong West Street 75
19: 68 Orchard Road
20: 68 Orchard Road
21: 442 Orchard Road
22: 308 Anchorvale Road
23: 620 Hougang Avenue 8
24: 21 Hougang Street 51
25: 348 Bedok Road
26: 946 Margaret Drive
27: 455 Sengkang West Avenue
28: 308 Anchorvale Road
29: 33 Sengkang West Avenue
30: 33 Sengkang West Avenue
31: 308 Anchorvale Road
32: 71 Pioneer Road
33: 71 Pioneer Road
34: 455 Sengkang West Avenue
35: 417 Fernvale Link
36: 4000 Jalan Bukit Merah
37: 8 Marina Boulevard
38: 237 Serangoon Avenue 3
39: 298 Tiong Bahru Road
40: 50 Havelock Road
41: 348 B

323: 9 Bishan Place
324: 164 Tampines Street 12
325: 208B New Upper Changi Road
326: 311 New Upper Changi Road
327: 201 Victoria Street
328: 10 Jurong East Street 12
329: 500 Lorong 6 Toa Payoh
330: 9 Bishan Place
331: 60 Paya Lebar Road
332: 200 Turf Club Road
333: 440 Bukit Batok West Avenue 8
334: 440 Bukit Batok West Avenue 8
335: 2 Choa Chu Kang Loop
336: 4 Bukit Batok Street 41
337: 444 Pasir Ris Drive 6
338: 1 Sengkang Square
339: 345 Jurong East Street 31
340: 51 Ang Mo Kio Avenue 3
341: 118 Rivervale Drive
342: 441 Pasir Ris Drive 6
343: 111 Somerset Road
344: 9 Tampines Street 32
345: 3 Gateway Drive
346: 118 Rivervale Drive
347: 1 Jelebu Road
348: 391 Orchard Rd
349: 118 Rivervale Drive
350: 437 Fernvale Road
351: 3 Gateway Drive
352: 6 Raffles Boulevard
353: 5 Changi Business Park Central 1
354: 823 Tampines Street 81
355: 5 Changi Business Park Central 1
356: 33 Sengkang West Avenue
357: 500 Dover Road
358: 10 Bayfront Avenue
359: 180 Kitchener Road
360: 68 Orchard Road
36

640: 7 Maxwell Road
641: 930 Yishun Avenue 2
642: 51 Yishun Central 1
643: 20 Marsiling Lane
644: 53 Boat Quay
645: 1 Jurong West Central
646: 7 Maxwell Road
647: 17 Upper Boon Keng Road
648: 26A Kallang Place
649: 105 Hougang Avenue 1
650: 1382 Ang Mo Kio Avenue 1
651: 93 Stamford Road
652: 6 Raffles Boulevard
653: 22 Keong Saik Road
654: 3 New Bugis Street
655: 1 Keong Saik Road
656: 23 Serangoon Central
657: 23 Serangoon Central
658: 71 Sungei Kadut Drive
659: 273C Punggol Place
660: 50 Hougang Avenue 1
661: 1 Rochor Canal Road
662: 145 Marine Parade Road
663: 363 Joo Chiat Road
664: 30 Merchant Road
665: 8 Maju Avenue
666: 95 Syed Alwi Road
667: 5001 Beach Road
668: 83 Geylang Road
669: 10 Tebing Lane
670: 26A Kallang Place
671: 10 Tebing Lane
672: 3 Simei Street 6
673: 3 Simei Street 6
674: 1 Pasir Ris Close
675: 10 Tampines Central 1
676: 1 Pasir Ris Close
677: 1 Pasir Ris Close
678: 11 Bedok North Street 1
679: 60 Paya Lebar Road
680: 5 Straits View
681: 311 New Upper Changi Roa

976: 9 Lorong 27A Geylang
977: 501 Orchard Road
978: 78 Airport Boulevard
979: 56 Jalan Kembangan
980: 80 Changi Boulevard
981: 65 Airport Boulevard
982: 1 Geylang Serai
983: 304 Orchard Road
984: 83 Punggol Central
985: 60 Nanyang Drive
986: 304 Orchard Road
987: 23 Serangoon Central
988: 118 Holland Avenue
989: 1 Pasir Ris Close
990: 60 Paya Lebar Road
991: 10 Kallang Road
992: 1 HarbourFront Walk
993: 612 Geylang Lorong 38
994: 304 Orchard Road
995: 18 Marina Gardens Drive
996: 1 Pasir Ris Central Street 3
997: 18 Marina Gardens Drive
998: 53 Ang Mo Kio Avenue 3
999: 1 HarbourFront Walk
1000: 145 Syed Alwi Road
1001: 78 Airport Boulevard
1002: 304 Orchard Road
1003: 51 Old Airport Road
1004: 206 Bedok North Street 1
1005: 4A Jalan Batu
1006: 120 Pasir Ris Central
1007: 304 Orchard Road
1008: 1 Park Road
1009: 6 Eunos Crescent
1010: 1 Sengkang Square
1011: 435 Orchard Road
1012: 530 Ang Mo Kio Avenue 10
1013: 391A Orchard Road
1014: 4A Jalan Batu
1015: 260 Orchard Road
1016: 545 Orch

1296: 418 Yishun Avenue 11
1297: 9 Bishan Place
1298: 1 Geylang Serai
1299: 9 Bishan Place
1300: 505 Jurong West St 52
1301: 2 Jurong East Street 21
1302: Yung Sheng Road
1303: 2 Jurong East Street 21
1304: 2 Jurong East Street 21
1305: 1 Jelebu Road
1306: 17 Petir Road
1307: 1 Jurong West Central 2
1308: 50 Jurong Gateway Road
1309: Yung Sheng Road
1310: 1 Jurong West Central 2
1311: 80 Marine Parade
1312: 1 Jurong West Central 2
1313: Yung Sheng Road
1314: 162 Ang Mo Kio Avenue 4
1315: 505 Jurong West St 52
1316: 100 Tras Street


,Date,Time,Location,Sub-location,Source,Notes,Longitude,Latitude,Postal
0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.657832233969,1.31133149338762,628505
1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.755832339467,1.34175978313366,659545
2,2021-06-01,1955h - 2030h,Seletar Mall (33 Sengkang West Avenue),NTUC FairPrice,https://mothership.sg/2021/06/june-4-covid-19-...,Added/ Updated on 2021-06-05,103.876156956828,1.3913688093851,797653
3,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01,103.903623796196,1.37791506138101,537476
4,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02,103.887847089168,1.37931783471773,538719
...,...,...,...,...,...,...,...,...,...
1312,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.706685012654,1.33945200436318,NIL
1313,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.721386502056,1.33414175848059,NIL
1314,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.838900879239,1.37406230373928,560162
1315,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.718447549568,1.34964230218296,640505


In [6]:
df.to_csv('data/Covid-Public-SG-LngLat.csv', index = False)

## Getting planning areas

In [7]:
lng_lat_df = pd.read_csv('data/Covid-Public-SG-LngLat.csv')
lng_lat_df

,Date,Time,Location,Sub-location,Source,Notes,Longitude,Latitude,Postal
0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.657832,1.311331,628505
1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.755832,1.341760,659545
2,2021-06-01,1955h - 2030h,Seletar Mall (33 Sengkang West Avenue),NTUC FairPrice,https://mothership.sg/2021/06/june-4-covid-19-...,Added/ Updated on 2021-06-05,103.876157,1.391369,797653
3,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01,103.903624,1.377915,537476
4,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02,103.887847,1.379318,538719
...,...,...,...,...,...,...,...,...,...
1312,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.706685,1.339452,NIL
1313,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.721387,1.334142,NIL
1314,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.838901,1.374062,560162
1315,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.718448,1.349642,640505


In [8]:
# Obtained from https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore
planning_area_list = ['Ang Mo Kio', 'Bedok', 'Bishan', 'Boon Lay', 'Bukit Batok', 'Bukit Merah', 'Bukit Panjang', 'Bukit Timah', 'Central Water Catchment', 'Changi Airport', 'Changi Bay', 'Choa Chu Kang', 'Clementi', 'Downtown Core', 'Geylang', 'Hougang', 'Jurong East', 'Jurong West', 'Kallang', 'Lim Chu Kang', 'Mandai', 'Marina East', 'Marina South', 'Marine Parade', 'Museum', 'Newton', 'North-Eastern Islands', 'Novena', 'Orchard', 'Outram', 'Pasir Ris', 'Paya Lebar', 'Pioneer', 'Punggol', 'Queenstown', 'River Valley', 'Rochor', 'Seletar', 'Sembawang', 'Sengkang', 'Serangoon', 'Simpang', 'Singapore River', 'Southern Islands', 'Straits View', 'Sungei Kadut', 'Tampines', 'Tanglin', 'Tengah', 'Toa Payoh', 'Tuas', 'Western Islands', 'Western Water Catchment', 'Woodlands', 'Yishun']

In [9]:
planning_area_df = pd.DataFrame({'Area': planning_area_list})
planning_area_df.head()

,Area
0,Ang Mo Kio
1,Bedok
2,Bishan
3,Boon Lay
4,Bukit Batok


In [10]:
planning_area_path = 'data/SG-Planning-Area.csv'

if os.path.isfile(planning_area_path):
    planning_area_df = pd.read_csv(planning_area_path)
else:
    for index, row in planning_area_df.iterrows():
        response = requests.get(request_url, params={
            'searchVal': row['Area'],
            'returnGeom': 'Y',
            'getAddrDetails': 'N'
        })
#         print('{}: {}'.format(index, row['Area']))
        if len(response.json()['results']) <= 0:
            continue
        result = response.json()['results'][0]
        planning_area_df.at[index,'Longitude']=result['LONGITUDE']
        planning_area_df.at[index,'Latitude']=result['LATITUDE']

    planning_area_df = planning_area_df.dropna()

    planning_area_df.to_csv(planning_area_path, index=False)

planning_area_df.head()

,Area,Longitude,Latitude
0,Ang Mo Kio,103.837993,1.368750
1,Bedok,103.953607,1.325015
2,Bishan,103.852048,1.347522
3,Boon Lay,103.705845,1.338493
4,Bukit Batok,103.753493,1.343794


In [11]:
baseline = planning_area_df.iloc[0]

def find_place(lng, lat):
    result = baseline['Area']
    
    baseline_a = np.array((float(baseline['Longitude']), float(baseline['Latitude'])))
    b = np.array((lng, lat))
    
    minimum_distance = np.linalg.norm(baseline_a-b)
    
    for index, row in planning_area_df.iterrows():
        a = np.array((float(row['Longitude']), float(row['Latitude'])))
        distance = np.linalg.norm(a-b)
        if distance < minimum_distance:
            minimum_distance = distance
            result = row['Area']
    
    return result

In [12]:
for index, row in lng_lat_df.iterrows():
    lng_lat_df.at[index,'Area']=find_place(row['Longitude'], row['Latitude'])
lng_lat_df

,Date,Time,Location,Sub-location,Source,Notes,Longitude,Latitude,Postal,Area
0,2021-06-01,1000h - 1230h,JIC Inspection Services (53 Pioneer Road),NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.657832,1.311331,628505,Tuas
1,2021-06-01,1100h - 1230h,VICOM Inspection Centre (511 Bukit Batok Stree...,NaN,https://www.todayonline.com/singapore/2-commun...,Added/ Updated on 2021-06-04,103.755832,1.341760,659545,Bukit Batok
2,2021-06-01,1955h - 2030h,Seletar Mall (33 Sengkang West Avenue),NTUC FairPrice,https://mothership.sg/2021/06/june-4-covid-19-...,Added/ Updated on 2021-06-05,103.876157,1.391369,797653,Seletar
3,2021-05-30,0940h - 1630h,NTUC Foodfare (476D Upper Serangoon View),NaN,https://www.todayonline.com/singapore/resorts-...,Added/ Updated on 2021-06-01,103.903624,1.377915,537476,Sengkang
4,2021-05-30,1100h - 2200h,Hougang Green Shopping Mall (21 Hougang Street...,Pizza Hut,https://www.todayonline.com/singapore/plaza-si...,Added/ Updated on 2021-06-02,103.887847,1.379318,538719,Sengkang
...,...,...,...,...,...,...,...,...,...,...
1312,2020-05-20,1630h - 1800h,Jurong Point (1 Jurong West Central 2),NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.706685,1.339452,NIL,Boon Lay
1313,2020-05-20,1900h - 2000h,Taman Jurong Shopping Centre (339 Yung Sheng R...,NTUC Fairprice,https://www.channelnewsasia.com/news/singapore...,NaN,103.721387,1.334142,NIL,Boon Lay
1314,2020-05-19,1045h - 1115h,Mayflower Market and Food Centre (162 Ang Mo K...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.838901,1.374062,560162,Ang Mo Kio
1315,2020-05-17,0900h - 0930h,Jurong West 505 Market and Food Centre (505 Ju...,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,103.718448,1.349642,640505,Boon Lay


In [13]:
lng_lat_df.to_csv('data/Covid-Public-SG-LngLat-Area.csv')

## Using postal code to get region 
Does not work because a lot of NIL postal codes

In [14]:
# sg_postal_df = pd.read_csv('data/SG-postal.csv')
# sg_postal_df.columns = sg_postal_df.columns.str.strip()
# intermediate_sg_postal_df = pd.concat([sg_postal_df['Postal Sector (1st 2 digits of postal codes)'].str.split(',', expand=True), 
#                  sg_postal_df['General Location'], 
#                  sg_postal_df['Postal District']], 
#                 axis=1)
# intermediate_sg_postal_df
# lookup_sg_postal_df = pd.melt(intermediate_sg_postal_df,id_vars=['Postal District','General Location'],value_name='Postal Sector').dropna().drop(columns=['variable'])


# lookup_sg_postal_df.sort_values("Postal Sector")

In [15]:
# df = df.replace("NIL", None)
# df['Postal-sector'] = df['Postal'].astype(str).str[:2]
# result_df = pd.merge(df, lookup_sg_postal_df, left_on="Postal-sector", right_on="Postal Sector", how="left")
# result_df

In [16]:
# result_df.to_csv('dtata/Covid-Public-SG-LngLat-Postal.csv')

In [17]:
# lng_lat_df = pd.read_csv('data/Covid-Public-SG-LngLat.csv')
# lng_lat_df['Postal'][lng_lat_df['Postal'] == "NIL"].count()